## KR-WordRank: method for word / keyword extraction

KR-WordRank는 Kim et al.(2014)^[1]의 논문을 바탕으로 한 비지도학습기반 단어 추출 기법으로, 데이터기반으로 주요단어 (키워드)를 추출하는 알고리즘이다. 하나의 도메인에 대한 문서들을 바탕으로 명사/형용사/동사/부사 (L set) 중에서 빈도수가 높거나, 주요 단어들과 함께 등장하는 단어를 키워드로 추출한다. KR-WordRank는 이름에서 나타나는바와 같이 단어 후보 (subtokens)을 이용하여 word-graph를 생성한 뒤, PageRank의 랭킹 학습 방식을 이용하여 word-graph의 hub subtokens을 추출한다. 

KR-WordRank는 다음의 가정을 기반으로 단어를 추출한다. **단어 주변에는 단어가 등장하며, 올바른 단어는 주위의 많은 단어들과 연결되어 있다. 그렇기 때문에 단어는 주위 단어들에 의하여 단어 점수가 보강(reinforced)된다.**


![kr_wordrank_structure](figs/kr_wordrank_fig1.png)


한국어는 의미를 지니는 단어 집합과 문법 기능을 하는 복합형태소 집합으로 나뉘어지며, [문법/명사] + [을/조사]와 같이 어절의 왼쪽에 의미를 지니는 단어인 명사/형용사/동사가 위치한다. 부사는 그 자체로 한 어절을 이룬다. 그렇기 때문에 KR-WordRank는 의미있는 단어로서 어절 자체나 어절의 왼쪽에 등장하는 L set을 추출한다. 또한 한국어는 한 글자에 지나치게 많은 의미가 담겨져 있어 해석이 모호하기 때문에 1음절 단어는 추출되는 단어에서 제외한다. 실제로 subtokens으로 이뤄진 word-graph에서 1음절 단어들은 매우 높은 랭킹을 지닌다. KR-WordRank는 아래 그림과 같이 subtokens을 어절의 위치에 따라 L/R tags를 부여하여 word-graph를 만든 뒤, 랭킹을 계산한다. 

![kr_wordrank_structure](figs/kr_wordrank_fig2.png)

논문에서 기술되지 않은 후처리(post-processing)가 추가되었다. 영화리뷰의 경우, '영화', '영화가', '영화를' 와 같이 "단어 + R set"이 함께 키워드로 추출된다. 이는 KR-WordRank가 주요 L set 혹은 어절을 추출하기 때문이며, '영화', '영화가' 주변 모두 올바른 단어가 위치하기 때문이다. 그렇기 때문에 '영화'라는 단어가 '영화가', '영화를' 등보다 높은 랭킹을 지녔다면, '영화' + R set는 L + R 복합어라 판단하여 제외하였다. 

        keywords = self._select_keywords(lset, rset)

두번째 후처리로, '영화', '음악', '영화음악'이 키워드로 추출되었고, '영화', '음악'이 모두 '영화음악'보다 랭킹이 높을 경우, '영화음악'은 합성어로 판단하여 이를 제거하였다. 

        keywords = self._filter_compounds(keywords)

마지막 후처리로, '스토리'가 상위 랭킹이 될 경우, 한 글자가 랭킹이 높아서 '스토' 역시 키워드로 추출될 수 있다. '스토리'가 상위 랭킹이 된다면 '스토'와 같은 substring은 키워드에서 제거하였다. 

        keywords = self._filter_subtokens(keywords)

사용법은 아래의 예제 코드와 같다. 

[1] Kim, H. J., Cho, S., & Kang, P. (2014). KR-WordRank: An Unsupervised Korean Word Extraction Method Based on WordRank. Journal of Korean Institute of Industrial Engineers, 40(1), 18-33.

In [1]:
import glob
import json
import sys

YOUR_GIT_REPOSITORY = '../../../soy/'
sys.path.append(YOUR_GIT_REPOSITORY)

단어 추출에 영어/숫자를 포함할 예정이라면 normalize함수를 이용하여 텍스트를 normalize할 것

In [2]:
def get_texts_scores(fname):
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f]
        docs = [doc for doc in docs if len(doc) == 2]
        
        if not docs:
            return [], []
        
        texts, scores = zip(*docs)
        return list(texts), list(scores)

# La La Land
fname = '../../data/naver_movie/comments/134963.txt'
texts, scores = get_texts_scores(fname)

In [ ]:
with open('../../data/naver_movie/comments/134963_norm.txt', 'w', encoding='utf-8') as f:
    for text, score in zip(texts, scores):
        text = normalize(text, english=True, number=True)
        f.write('%s\t%s\n' % (text, str(score)))

In [3]:
# La La Land
fname = '../../data/naver_movie/comments/134963_norm.txt'
texts, scores = get_texts_scores(fname)

In [8]:
min_count = 5   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
kr_wordrank = KR_WordRank(min_count, max_length)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10
verbose = True
keywords, rank, graph = kr_wordrank.extract(texts, beta, max_iter, verbose)

scan vocabs ... 
num vocabs = 15097
done


In [ ]:
keywords, rank, graph = kr_wordrank.extract(texts, beta, max_iter, verbose, vocabulary={}, bias={})

위와 같이 vocabulary를 미리 설정하거나 decaying factor를 단어별로 다르게 (bias) 할당할 수 있으며, 모든 단어의 랭킹의 총 합은 vocabulary size와 같음. 즉 default decaying factor는 1.0

In [9]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:100]:
    print('%8s:\t%.4f' % (word, r))

      영화:	229.7889
     관람객:	112.3404
      너무:	78.4055
      음악:	37.6247
      정말:	37.2504
     마지막:	34.9952
      최고:	22.4425
      사랑:	21.1355
     뮤지컬:	20.7357
      꿈을:	19.5282
     여운이:	19.4032
      보고:	19.4005
      아름:	18.6495
      진짜:	18.5599
      영상:	18.1099
      좋았:	17.8625
      노래:	16.9019
     스토리:	16.2600
      좋은:	15.4661
      그냥:	15.2136
      현실:	15.0772
      생각:	14.6264
      인생:	14.2642
      좋고:	13.9971
      지루:	13.8732
      다시:	13.7812
      감동:	13.4817
      느낌:	12.3127
      ㅠㅠ:	12.1447
      좋아:	11.9586
      보는:	11.8995
      계속:	11.5078
      조금:	11.4933
      연기:	11.4792
      많이:	11.0381
     그리고:	11.0165
      장면:	11.0128
      있는:	10.7771
      하는:	10.5781
      결말:	10.5147
      재미:	10.2599
      남는:	10.0084
      사람:	9.8278
      재밌:	9.8230
      처음:	9.5475
      모두:	9.4312
     봤는데:	9.2878
     하지만:	8.9914
     라이언:	8.7217
     눈물이:	8.7091
      내내:	8.7002
      연출:	8.5802
      모든:	8.5401
      이런:	8.4710
      재즈:	8.2790
      of:	8.1979
    

## PageRank를 이용한 WordRank 구현, KR-WordRank와의 비교

In [3]:
from soy.ml.graph import Graph, PageRank

fname = '../../data/naver_movie/comments/134963_norm.txt'
texts, scores = get_texts_scores(fname)

### 띄어쓰기 정보를 이용하지 않는 WordRank의 경우

In [ ]:
texts_wo_space = [text.replace(' ','') for text in texts]

In [10]:
min_count = 5   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이

def extract_links(text):
    links = []
    len_text = len(text)
    for bl in range(0, len_text):
        for rl in range(1, min(max_length, len_text) + 1):
            br = bl + rl
            if br >= len_text:
                continue
            for rr in range(1, min(max_length, len_text) + 1):
                er = br + rr
                if er <= len_text:
                    links.append((text[bl:br], text[br:er]))
    return links
     
#extract_links('abcde')

In [13]:
# 노드 출현 빈도수 확인

from collections import defaultdict
counter = defaultdict(lambda: 0)

for text in texts_wo_space:
    if not text:
        continue
    for node_l, node_r in extract_links(text):
        counter[node_l] += 1
        counter[node_r] += 1

counter = {node:freq for node, freq in counter.items() if freq >= min_count}

In [ ]:
# PageRank에 입력될 그래프 만들기

graph = Graph()
for text in texts_wo_space:
    if not text:
        continue
    for node_l, node_r in extract_links(text):
        if (not node_l in counter) or (not node_r in counter):
            continue
        graph.add(node_l, node_r, undirected=True)

pagerank = PageRank(graph)
ranks = pagerank.train()